# Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [20]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:

                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices = np.random.permutation(np.arange(D))[:D_try]
                left, right = make_density_split_node(node, N, feature_indices)
                stack += [left, right]
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)

    def predict(self, x):
        m, M = self.box
        
        print(x, m, M)
        if np.any(x > M) or np.any(x < m):
            return 0

        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        return leaf.response * self.prior

In [4]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    
    for feat in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        # It is necessary because otherwise we would recieve a threshold candidates
        # on a feature value not between two of them
        data_unique = np.sort(np.unique(node.data[:, feat]), axis=0)
        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2
        
        for t in tj:
            loo_error = ( leave_one_out_error(node.data, node.data[node.data[:, feat] <= t], t, N)
                        + leave_one_out_error(node.data, node.data[node.data[:, feat] > t], t, N))

            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                node.feature = feat
                node.threshold = t

    # create children
    node.left = Node()
    node.right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    is_left = node.data[:, node.feature] < node.threshold
    node.left.data = node.data[is_left]
    node.left.box = (node.left.data.min(axis=0), node.left.data.max(axis=0))
    node.right.data = node.data[~is_left]
    node.right.box = (node.right.data.min(axis=0), node.right.data.max(axis=0))
    
    return node.left, node.right


def leave_one_out_error(data, data_below_threshold, t, N):
    N_l = data_below_threshold.shape[0]
    
    M_l = data_below_threshold.max(axis=0)
    m_l = data_below_threshold.min(axis=0)
    
    # sadly this also works to fix the box of the node
    M_l += M_l == m_l

    V_l = np.product(M_l - m_l)
    return (N_l / (N * V_l)) * ((N_l / N) - 2 * (N_l - 1) / (N - 1))


In [5]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    node.response = n / N

# Decision Tree

In [6]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices = np.random.permutation(np.arange(D))[:D_try]
                left, right = make_decision_split_node(node, feature_indices)
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response

In [7]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    node.feature = None
    node.threshold = None
    
    for feat in feature_indices:
        data_unique = np.sort(np.unique(node.data[:, feat]), axis=0)

        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2
        
        for t in tj:
            gi_error = (gini_impurity(node.labels[node.data[:, feat] <= t])
                      + gini_impurity(node.labels[node.data[:, feat] > t]))
            
            if gi_error < e_min:
                e_min = gi_error
                node.feature = feat
                node.threshold = t
    
    # create children
    node.left = Node()
    node.right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above    
    is_left = node.data[:, node.feature] < node.threshold
    node.left.data = node.data[is_left]
    node.left.labels = node.labels[is_left]
    node.right.data = node.data[~is_left]
    node.right.labels = node.labels[~is_left]
    
    # return the children (to be placed on the stack)
    return node.left, node.right

def gini_impurity(labels):
    N_l = labels.shape[0]
    return N_l * (1 - np.sum(np.square((np.unique(labels)[:, None] == labels).sum(axis=1)) / (N_l ** 2)))

In [8]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = ...
    node.response = ... # your code here

In [9]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return len(np.unique(node.labels)) == 1

# Evaluation of Density and Decision Tree

In [10]:
# read and prepare the digits data
digits = load_digits()
data = digits.data
images = digits.images
target = digits.target
target_names = digits.target_names

In [11]:
# train trees, plot training error confusion matrices, and comment on your results
... # your code here

In [21]:
tree = DensityTree()
threes = data[target == 3]
tree.train(threes, len(threes) / len(data))

In [26]:
tree.predict(data[target != 3][3])

[ 0.  0.  0.  1. 11.  0.  0.  0.  0.  0.  0.  7.  8.  0.  0.  0.  0.  0.
  1. 13.  6.  2.  2.  0.  0.  0.  7. 15.  0.  9.  8.  0.  0.  5. 16. 10.
  0. 16.  6.  0.  0.  4. 15. 16. 13. 16.  1.  0.  0.  0.  0.  3. 15. 10.
  0.  0.  0.  0.  0.  2. 16.  4.  0.  0.] [0. 0. 0. 6. 2. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6. 1. 0. 0. 0.] [ 0.  6. 16. 16. 16. 16. 15.  1.  1. 14. 16. 16. 16. 16. 16.  2.  1. 15.
 16. 16. 16. 16.  9.  0.  0.  8. 16. 16. 16. 16.  3.  0.  0.  5. 13. 16.
 16. 16. 11.  0.  0. 11. 16. 16. 16. 16. 16.  0.  0. 13. 16. 16. 16. 16.
 16.  5.  0.  5. 16. 16. 16. 16. 16.  8.]


0

In [ ]:
x = data[target == 3][0]
(not (x.max() > 23)) * (not (x.min() < 2))

In [ ]:
tree = DecisionTree()
tree.train(data, target)

# Density and Decision Forest

In [ ]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [ ]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here